# 3주차 과제: Retriever와 RetrievalQA 체인으로 기술 문서 Q&A 챗봇 만들기

## 개요
- 2주차에서 구현한 Document Loader → Text Splitter → Embedding → Vector Store 파이프라인 활용
- Retriever와 RetrievalQA 체인을 통한 완전한 RAG 챗봇 구현
- 검색된 문서를 기반으로 자연어 답변 생성

## Part 1: 환경 설정 및 2주차 파이프라인 재구성

In [1]:
# ===============================================
# 환경 변수 및 설정
# ===============================================
import os
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    print(" * 경고: OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")
else:
    # API 키의 처음 10자만 표시 (보안)
    api_key = os.getenv("OPENAI_API_KEY")
    print(f" * OpenAI API 키 로드 완료 !")


# 파일 경로 설정
DATA_DIRECTORY = r"D:\data"  # 문서가 저장된 디렉토리
PERSIST_DIRECTORY = r"D:\data\chroma_db"  # 벡터 DB 저장 경로

# Retriever 설정 파라미터 (쉽게 수정 가능)
RETRIEVER_CONFIG = {
    "search_type": "similarity",  # similarity, mmr, similarity_score_threshold
    "search_kwargs": {
        "k": 3,  # 검색할 문서 수
        # "score_threshold": 0.5  # similarity_score_threshold 사용 시
    },
}

# LLM 설정
LLM_CONFIG = {"model": "gpt-5-mini", "temperature": 1, "max_tokens": 1000}

print("환경 설정 완료")
print(f"- 데이터 디렉토리: {DATA_DIRECTORY}")
print(f"- 벡터 DB 경로: {PERSIST_DIRECTORY}")
print(f"- Retriever 설정: {RETRIEVER_CONFIG}")
print(f"- LLM 설정: {LLM_CONFIG}")

 * OpenAI API 키 로드 완료 !
환경 설정 완료
- 데이터 디렉토리: D:\data
- 벡터 DB 경로: D:\data\chroma_db
- Retriever 설정: {'search_type': 'similarity', 'search_kwargs': {'k': 3}}
- LLM 설정: {'model': 'gpt-5-mini', 'temperature': 1, 'max_tokens': 1000}


In [2]:
# ===============================================
# 유틸리티 함수 정의 (2주차 코드)
# ===============================================


def load_documents_safe(directory_path, file_types=None, verbose=True):
    """디렉토리에서 문서를 안전하게 로드하는 함수"""
    from langchain_community.document_loaders import (
        PyPDFLoader,
        Docx2txtLoader,
        TextLoader,
        UnstructuredMarkdownLoader,
        UnstructuredHTMLLoader,
    )
    import os

    if file_types is None:
        file_types = [".pdf", ".docx", ".txt", ".md", ".html"]

    documents = []
    error_files = []

    loader_map = {
        ".pdf": PyPDFLoader,
        ".docx": Docx2txtLoader,
        ".txt": TextLoader,
        ".md": UnstructuredMarkdownLoader,
        ".html": UnstructuredHTMLLoader,
    }

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_ext = os.path.splitext(file)[1].lower()

            if file_ext not in file_types:
                continue

            try:
                loader_class = loader_map.get(file_ext)
                if loader_class:
                    if file_ext == ".txt":
                        loader = loader_class(file_path, encoding="utf-8")
                    else:
                        loader = loader_class(file_path)

                    docs = loader.load()

                    # 메타데이터 추가
                    for doc in docs:
                        doc.metadata["source"] = file_path
                        doc.metadata["file_name"] = file
                        doc.metadata["file_type"] = file_ext

                    documents.extend(docs)

                    if verbose:
                        print(f"  ✓ {file} 로드 완료 (문서 {len(docs)}개)")

            except Exception as e:
                error_files.append((file, str(e)))
                if verbose:
                    print(f"  ✗ {file} 로드 실패: {e}")

    return documents, error_files


def split_documents_optimized(
    documents, chunk_size=1000, chunk_overlap=200, separators=None
):
    """문서를 최적화된 방식으로 분할하는 함수"""
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    if separators is None:
        separators = ["\n\n", "\n", ".", "!", "?", ",", " ", ""]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=separators,
        length_function=len,
    )

    chunks = []
    for i, doc in enumerate(documents):
        doc_chunks = text_splitter.split_documents([doc])

        # 청크에 추가 메타데이터 추가
        for j, chunk in enumerate(doc_chunks):
            chunk.metadata["chunk_index"] = j
            chunk.metadata["total_chunks"] = len(doc_chunks)
            chunk.metadata["doc_index"] = i

        chunks.extend(doc_chunks)

    return chunks


def create_vectorstore_safe(chunks, embeddings, persist_dir, batch_size=100):
    """안전하게 벡터 저장소를 생성하는 함수"""
    import os
    import shutil
    from langchain_community.vectorstores import Chroma

    # 기존 DB가 있으면 삭제 (차원 불일치 문제 방지)
    if os.path.exists(persist_dir):
        print(f"  기존 벡터 DB 삭제 중...")
        shutil.rmtree(persist_dir)

    # 디렉토리 생성
    os.makedirs(persist_dir, exist_ok=True)

    vectorstore = None
    total_chunks = len(chunks)

    for i in range(0, total_chunks, batch_size):
        batch = chunks[i : i + batch_size]
        batch_end = min(i + batch_size, total_chunks)
        print(f"  처리 중: {i+1}-{batch_end}/{total_chunks}")

        if vectorstore is None:
            vectorstore = Chroma.from_documents(
                documents=batch,
                embedding=embeddings,
                persist_directory=persist_dir,
                collection_metadata={"hnsw:space": "cosine"},
            )
        else:
            vectorstore.add_documents(batch)

    if vectorstore:
        vectorstore.persist()

    return vectorstore


print("유틸리티 함수 정의 완료")

유틸리티 함수 정의 완료


## Part 2: 2주차 파이프라인 실행 (데이터 준비)

In [3]:
# ===============================================
# 셀 4: 문서 로드
# ===============================================
print("[Step 1] 문서 로드 시작")
print(f"디렉토리: {DATA_DIRECTORY}\n")

# 데이터 디렉토리 생성 및 확인
os.makedirs(DATA_DIRECTORY, exist_ok=True)

# 디렉토리에 파일이 있는지 확인
import glob

files_in_dir = glob.glob(f"{DATA_DIRECTORY}/**/*", recursive=True)
print(f"데이터 폴더의 파일 목록:")
for f in files_in_dir:
    if os.path.isfile(f):
        print(f"  - {f}")

if not files_in_dir:
    print("* data 폴더가 비어있습니다! PDF, DOCX, TXT 등의 파일을 추가하세요.")

documents, error_files = load_documents_safe(
    DATA_DIRECTORY, file_types=[".pdf", ".docx", ".txt", ".md", ".html"], verbose=True
)

print(f"\n로드 완료: 총 {len(documents)}개 문서")
if error_files:
    print(f"로드 실패: {len(error_files)}개 파일")

[Step 1] 문서 로드 시작
디렉토리: D:\data

데이터 폴더의 파일 목록:
  - D:\data\2023 당뇨병 진료지침_전문_240620.pdf
  - D:\data\rag_experiment_20251201_175820.json
  - D:\data\chroma_db\chroma.sqlite3
  - D:\data\chroma_db\faef8aa0-50df-43a2-a96c-9ba7156fcb43\data_level0.bin
  - D:\data\chroma_db\faef8aa0-50df-43a2-a96c-9ba7156fcb43\header.bin
  - D:\data\chroma_db\faef8aa0-50df-43a2-a96c-9ba7156fcb43\index_metadata.pickle
  - D:\data\chroma_db\faef8aa0-50df-43a2-a96c-9ba7156fcb43\length.bin
  - D:\data\chroma_db\faef8aa0-50df-43a2-a96c-9ba7156fcb43\link_lists.bin
  ✓ 2023 당뇨병 진료지침_전문_240620.pdf 로드 완료 (문서 428개)

로드 완료: 총 428개 문서


In [4]:
# ===============================================
# 셀 5: 문서 분할
# ===============================================
print("[Step 2] 문서 분할")

chunks = split_documents_optimized(documents, chunk_size=1000, chunk_overlap=200)

print(f"분할 완료: 총 {len(chunks)}개 청크 생성")
print(
    f"평균 청크 크기: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f} 문자"
)

[Step 2] 문서 분할
분할 완료: 총 925개 청크 생성
평균 청크 크기: 742 문자


In [5]:
# ===============================================
# 셀 6: 임베딩 모델 설정
# ===============================================
print("[Step 3] 임베딩 모델 로드")

from langchain_community.embeddings import HuggingFaceEmbeddings

# 한국어 특화 임베딩 모델 사용
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

# 임베딩 차원 확인
test_embedding = embeddings.embed_query("테스트")
print(f"임베딩 모델 로드 완료")
print(f"임베딩 차원: {len(test_embedding)}")

[Step 3] 임베딩 모델 로드


C:\Users\hfdt\AppData\Local\Temp\ipykernel_944\4159656875.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


임베딩 모델 로드 완료
임베딩 차원: 768


In [6]:
# ===============================================
# 셀 7: 벡터 DB 생성
# ===============================================
print("[Step 4] 벡터 DB 생성")

# 기존 벡터스토어가 있으면 먼저 정리
if "vectorstore" in globals():
    try:
        # 기존 연결 종료
        if hasattr(vectorstore, "_client"):
            vectorstore._client.close()
        del vectorstore
        print("  기존 벡터스토어 연결 종료")
    except:
        pass

# 잠시 대기 (Windows 파일 시스템 동기화)
import time
import gc

gc.collect()  # 가비지 컬렉션 강제 실행
time.sleep(1)

# 안전한 삭제 시도
if os.path.exists(PERSIST_DIRECTORY):
    try:
        import shutil

        shutil.rmtree(PERSIST_DIRECTORY)
        print(f"  기존 벡터 DB 삭제 완료")
    except PermissionError:
        # 삭제 실패 시 새로운 디렉토리 사용
        import datetime

        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        PERSIST_DIRECTORY = f"./chroma_db_korean_{timestamp}"
        print(f"  * 기존 DB 삭제 실패. 새 디렉토리 사용: {PERSIST_DIRECTORY}")

# 벡터 DB 생성
vectorstore = create_vectorstore_safe(
    chunks, embeddings, PERSIST_DIRECTORY, batch_size=50
)

print(f"\n벡터 DB 생성 완료")
print(f"  - 저장 위치: {PERSIST_DIRECTORY}")
print(f"  - 저장된 문서 수: {len(chunks)}")

[Step 4] 벡터 DB 생성
  * 기존 DB 삭제 실패. 새 디렉토리 사용: ./chroma_db_korean_20251201_192847
  처리 중: 1-50/925
  처리 중: 51-100/925
  처리 중: 101-150/925
  처리 중: 151-200/925
  처리 중: 201-250/925
  처리 중: 251-300/925
  처리 중: 301-350/925
  처리 중: 351-400/925
  처리 중: 401-450/925
  처리 중: 451-500/925
  처리 중: 501-550/925
  처리 중: 551-600/925
  처리 중: 601-650/925
  처리 중: 651-700/925
  처리 중: 701-750/925
  처리 중: 751-800/925
  처리 중: 801-850/925
  처리 중: 851-900/925
  처리 중: 901-925/925

벡터 DB 생성 완료
  - 저장 위치: ./chroma_db_korean_20251201_192847
  - 저장된 문서 수: 925


C:\Users\hfdt\AppData\Local\Temp\ipykernel_944\2793410716.py:132: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## Part 3: Retriever 설정 (3주차 신규)

In [7]:
# ===============================================
# 셀 8: Retriever 생성 및 설정
# ===============================================
print("[Step 5] Retriever 설정")

# 기존 벡터 DB 로드 (이미 생성된 경우)
from langchain_community.vectorstores import Chroma

# 벡터스토어가 None이면 다시 로드
if vectorstore is None:
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings
    )
    print("  기존 벡터 DB 로드 완료")

# Retriever 생성 (설정 파라미터 사용)
retriever = vectorstore.as_retriever(
    search_type=RETRIEVER_CONFIG["search_type"],
    search_kwargs=RETRIEVER_CONFIG["search_kwargs"],
)

print(f"\nRetriever 설정 완료:")
print(f"  - 검색 타입: {RETRIEVER_CONFIG['search_type']}")
print(f"  - 검색 파라미터: {RETRIEVER_CONFIG['search_kwargs']}")

# Retriever 테스트
test_query = "LangChain"
test_docs = retriever.get_relevant_documents(test_query)
print(f"\n테스트 검색 '{test_query}': {len(test_docs)}개 문서 검색됨")

[Step 5] Retriever 설정

Retriever 설정 완료:
  - 검색 타입: similarity
  - 검색 파라미터: {'k': 3}

테스트 검색 'LangChain': 3개 문서 검색됨


C:\Users\hfdt\AppData\Local\Temp\ipykernel_944\2728291185.py:28: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  test_docs = retriever.get_relevant_documents(test_query)


## Part 4: RetrievalQA 체인 구현 (3주차 핵심)

In [8]:
# ===============================================
# 셀 9: LLM 및 프롬프트 템플릿 설정
# ===============================================
print("[Step 6] LLM 및 프롬프트 설정")

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# LLM 초기화
llm = ChatOpenAI(
    model=LLM_CONFIG["model"],
    temperature=LLM_CONFIG["temperature"],
    max_tokens=LLM_CONFIG["max_tokens"],
)

# 프롬프트 템플릿 정의
prompt_template = """당신은 주어진 문서에 기반해서만 답변하는 AI 어시스턴트입니다.

다음 규칙을 반드시 따라주세요:
1. 제공된 문서 내용을 기반으로만 답변하세요.
2. 문서에 없는 내용은 "제공된 문서에 해당 정보가 없습니다"라고 명확히 말씀해주세요.
3. 답변 시 근거가 된 문서의 출처를 명시해주세요.

참고 문서:
{context}

질문: {question}

답변 형식:
【답변】
(1줄 요약 답변을 먼저 제공)

【상세 설명】
(필요시 자세한 설명 추가)

【출처】
(근거가 된 문서 정보: 파일명, 청크 인덱스)

답변:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

print("LLM 및 프롬프트 설정 완료")
print(f"  - 모델: {LLM_CONFIG['model']}")
print(f"  - Temperature: {LLM_CONFIG['temperature']}")

[Step 6] LLM 및 프롬프트 설정
LLM 및 프롬프트 설정 완료
  - 모델: gpt-5-mini
  - Temperature: 1


In [9]:
# ===============================================
# 셀 10: RetrievalQA 체인 구현 (방법 1: LangChain RetrievalQA)
# ===============================================
print("[Step 7] RetrievalQA 체인 구현 (방법 1)")

from langchain.chains import RetrievalQA

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # stuff, map_reduce, refine, map_rerank
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT, "verbose": False},
)

print("RetrievalQA 체인 생성 완료 (LangChain 클래스 방식)")

[Step 7] RetrievalQA 체인 구현 (방법 1)
RetrievalQA 체인 생성 완료 (LangChain 클래스 방식)


In [10]:
# ===============================================
# 셀 11: RetrievalQA 체인 구현 (방법 2: LCEL 스타일)
# ===============================================
print("[Step 7-2] RetrievalQA 체인 구현 (방법 2: LCEL)")

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


def format_docs(docs):
    """검색된 문서를 포맷팅하는 함수"""
    formatted_docs = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get("file_name", "Unknown")
        chunk_idx = doc.metadata.get("chunk_index", "Unknown")
        content = doc.page_content[:500]  # 내용 일부만 표시

        formatted = f"""[문서 {i+1}]
파일: {source}
청크: {chunk_idx}
내용: {content}...
"""
        formatted_docs.append(formatted)

    return "\n".join(formatted_docs)


# LCEL 스타일 체인 구성
lcel_qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | PROMPT
    | llm
    | StrOutputParser()
)

print("LCEL 스타일 RetrievalQA 체인 생성 완료")

[Step 7-2] RetrievalQA 체인 구현 (방법 2: LCEL)
LCEL 스타일 RetrievalQA 체인 생성 완료


## Part 5: Q&A 챗봇 인터페이스

In [11]:
# ===============================================
# 셀 12: 챗봇 헬퍼 함수
# ===============================================


def answer_question_v1(question, qa_chain):
    """LangChain RetrievalQA를 사용한 답변 생성"""
    try:
        result = qa_chain({"query": question})

        answer = result["result"]
        source_docs = result.get("source_documents", [])

        # 출처 정보 추가
        if source_docs:
            sources = []
            for doc in source_docs[:3]:  # 상위 3개만
                file_name = doc.metadata.get("file_name", "Unknown")
                chunk_idx = doc.metadata.get("chunk_index", "Unknown")
                sources.append(f"- {file_name} (청크 {chunk_idx})")

            if "【출처】" not in answer:
                answer += "\n\n【출처】\n" + "\n".join(sources)

        return answer

    except Exception as e:
        return f"답변 생성 중 오류 발생: {str(e)}"


def answer_question_v2(question, lcel_chain):
    """LCEL 체인을 사용한 답변 생성"""
    try:
        answer = lcel_chain.invoke(question)
        return answer
    except Exception as e:
        return f"답변 생성 중 오류 발생: {str(e)}"


print("챗봇 헬퍼 함수 정의 완료")

챗봇 헬퍼 함수 정의 완료


In [12]:
# ===============================================
# 셀 13: 대화형 Q&A 챗봇 인터페이스
# ===============================================
print("=" * 50)
print("*** 기술 문서 Q&A 챗봇 ***")
print("=" * 50)
print("질문을 입력하세요. 종료하려면 'exit', 'quit', 또는 '종료'를 입력하세요.\n")

# 방법 선택
use_lcel = False  # True :  LCEL 체인 사용, False : RetrievalQA 사용

if use_lcel:
    print("[LCEL 체인 모드]\n")
    chain_to_use = lcel_qa_chain
    answer_func = answer_question_v2
else:
    print("[LangChain RetrievalQA 모드]\n")
    chain_to_use = qa_chain
    answer_func = answer_question_v1

# 대화 루프
conversation_history = []

while True:
    question = input("\n Q: 질문- ").strip()

    if question.lower() in ["exit", "quit", "종료", "q"]:
        print("\n 챗봇을 종료합니다. 감사합니다!")
        break

    if not question:
        print("* 질문을 입력해주세요.")
        continue

    print("\n * 관련 문서를 검색 중...")

    # 답변 생성
    if use_lcel:
        answer = answer_func(question, chain_to_use)
    else:
        answer = answer_func(question, chain_to_use)

    print("\n A: 답변 -")
    print("-" * 40)
    print(answer)
    print("-" * 40)

    # 대화 기록 저장
    conversation_history.append({"question": question, "answer": answer})

*** 기술 문서 Q&A 챗봇 ***
질문을 입력하세요. 종료하려면 'exit', 'quit', 또는 '종료'를 입력하세요.

[LangChain RetrievalQA 모드]


 * 관련 문서를 검색 중...


C:\Users\hfdt\AppData\Local\Temp\ipykernel_944\4038454400.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})



 A: 답변 -
----------------------------------------


【출처】
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 1)
----------------------------------------

 챗봇을 종료합니다. 감사합니다!


## Part 6: 테스트 시나리오 (최소 8문항)

In [13]:
# ===============================================
# 셀 14: 테스트 시나리오 정의
# ===============================================

# 테스트 질문 세트
test_questions = [
    # 카테고리 1: 문서에 답이 명확히 있는 질문 (5개)
    {
        "category": "명확한 답변",
        "question": "당뇨병의 진단 기준은 무엇인가요?",
        "expected": "공복혈당, 당화혈색소, 경구당부하검사 기준 설명",
    },
    {
        "category": "명확한 답변",
        "question": "당화혈색소(HbA1c)의 목표치는 얼마인가요?",
        "expected": "일반적으로 7% 미만, 개별화된 목표 설정",
    },
    {
        "category": "명확한 답변",
        "question": "메트포르민(Metformin)의 작용 기전과 용법은?",
        "expected": "1차 약제, 간 포도당 생성 억제, 용량 등",
    },
    {
        "category": "명확한 답변",
        "question": "당뇨병성 망막병증의 선별검사는 언제 시행하나요?",
        "expected": "제2형 당뇨병 진단 시, 제1형은 5년 후부터 매년",
    },
    {
        "category": "명확한 답변",
        "question": "저혈당의 정의와 증상은 무엇인가요?",
        "expected": "70mg/dL 미만, 떨림, 발한, 의식저하 등",
    },
    # 카테고리 2: 문서에 없거나 애매한 질문 (3개)
    {
        "category": "문서에 없음",
        "question": "2024년 노벨물리학상 수상자는 누구인가요?",
        "expected": "문서에 정보가 없다는 답변",
    },
    {
        "category": "문서에 없음",
        "question": "비트코인의 현재 가격은 얼마인가요?",
        "expected": "문서에 정보가 없다는 답변",
    },
    {
        "category": "문서에 없음",
        "question": "김치찌개 맛있게 끓이는 방법은?",
        "expected": "문서에 정보가 없다는 답변",
    },
    # 카테고리 3: 비교/응용형 질문 (2개)
    {
        "category": "비교/응용",
        "question": "SGLT2 억제제와 GLP-1 수용체 작용제의 차이점을 비교해주세요.",
        "expected": "작용 기전, 체중 감소, 심혈관 이익 등 비교",
    },
    {
        "category": "비교/응용",
        "question": "새로 진단받은 제2형 당뇨병 환자의 초기 치료 전략은 어떻게 수립하나요?",
        "expected": "생활습관 교정, 메트포르민, 개별화된 목표 설정 등",
    },
]

print(f"테스트 질문 {len(test_questions)}개 준비 완료")
print("\n카테고리별 분포:")
print("- 명확한 답변 가능: 5개")
print("- 문서에 없는 내용: 3개")
print("- 비교/응용형: 2개")

테스트 질문 10개 준비 완료

카테고리별 분포:
- 명확한 답변 가능: 5개
- 문서에 없는 내용: 3개
- 비교/응용형: 2개


In [14]:
# ===============================================
# 셀 15: 자동 테스트 실행
# ===============================================
print("=" * 60)
print("* 테스트 시나리오 실행")
print("=" * 60)

test_results = []

for i, test_case in enumerate(test_questions, 1):
    print(f"\n[테스트 {i}/{len(test_questions)}]")
    print(f"카테고리: {test_case['category']}")
    print(f"질문: {test_case['question']}")
    print("-" * 40)

    # 답변 생성
    answer = answer_func(test_case["question"], chain_to_use)

    # 답변 일부만 출력 (전체는 결과에 저장)
    print("답변 (요약):")
    print(answer[:300] + "..." if len(answer) > 300 else answer)

    # 결과 저장
    test_results.append(
        {
            "번호": i,
            "카테고리": test_case["category"],
            "질문": test_case["question"],
            "예상 답변": test_case["expected"],
            "실제 답변": answer,
            "답변 길이": len(answer),
        }
    )

    print("\n" + "=" * 60)

* 테스트 시나리오 실행

[테스트 1/10]
카테고리: 명확한 답변
질문: 당뇨병의 진단 기준은 무엇인가요?
----------------------------------------
답변 (요약):


【출처】
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)


[테스트 2/10]
카테고리: 명확한 답변
질문: 당화혈색소(HbA1c)의 목표치는 얼마인가요?
----------------------------------------
답변 (요약):


【출처】
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 1)
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)


[테스트 3/10]
카테고리: 명확한 답변
질문: 메트포르민(Metformin)의 작용 기전과 용법은?
----------------------------------------
답변 (요약):


【출처】
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 1)
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 1)
- 2023 당뇨병 진료지침_전문_240620.pdf (청크 1)


[테스트 4/10]
카테고리: 명확한 답변
질문: 당뇨병성 망막병증의 선별검사는 언제 시행하나요?
----------------------------------------
답변 (요약):
【답변】
1형 당뇨병은 진단 후 5년 이내, 2형 당뇨병은 진단과 동시에 망막 주변부를 포함한 안저검사 및 포괄적 안과검진을 시행하고, 그 이후는 보통 매년 검사하되 망막병증 소견이 없고 혈당조절이 양호하면 1–2년 간격으로 연장할 수 있습니다.

【상세 설명】
- 1형당뇨병: 당뇨병 진단 후 5년 이내에 망막 주변부를 포함한 안저검사 및 포괄적인 안과검진 권고.  
- 2형

In [17]:
# ===============================================
# 셀 16: 테스트 결과 분석 및 리포트
# ===============================================
import pandas as pd

print("\n" + "=" * 60)
print(" 테스트 결과 분석")
print("=" * 60)

# DataFrame으로 변환
df_results = pd.DataFrame(test_results)

# 기본 통계
print("\n1. 답변 통계:")
print(f"  - 총 테스트 수: {len(df_results)}")
print(f"  - 평균 답변 길이: {df_results['답변 길이'].mean():.0f} 문자")
print(f"  - 최소 답변 길이: {df_results['답변 길이'].min()} 문자")
print(f"  - 최대 답변 길이: {df_results['답변 길이'].max()} 문자")

# 카테고리별 분석
print("\n2. 카테고리별 평균 답변 길이:")
category_stats = df_results.groupby("카테고리")["답변 길이"].mean()
for cat, avg_len in category_stats.items():
    print(f"  - {cat}: {avg_len:.0f} 문자")

# "문서에 없음" 검출 분석
print("\n3. '문서에 없음' 답변 검출:")
no_info_keywords = ["문서에", "정보가 없", "찾을 수 없", "제공된 문서"]
for _, row in df_results[df_results["카테고리"] == "문서에 없음"].iterrows():
    has_no_info = any(keyword in row["실제 답변"] for keyword in no_info_keywords)
    status = " O 올바름" if has_no_info else "X 할루시네이션 의심"
    print(f"  - 질문 {row['번호']}: {status}")

# 결과 요약 테이블
print("\n4. 전체 테스트 결과 요약:")
summary_df = df_results[["번호", "카테고리", "질문", "답변 길이"]].copy()
summary_df["답변 요약"] = df_results["실제 답변"].str[:100] + "..."
print(summary_df.to_string(index=False))


 테스트 결과 분석

1. 답변 통계:
  - 총 테스트 수: 10
  - 평균 답변 길이: 224 문자
  - 최소 답변 길이: 117 문자
  - 최대 답변 길이: 516 문자

2. 카테고리별 평균 답변 길이:
  - 명확한 답변: 197 문자
  - 문서에 없음: 299 문자
  - 비교/응용: 180 문자

3. '문서에 없음' 답변 검출:
  - 질문 6:  O 올바름
  - 질문 7:  O 올바름
  - 질문 8: X 할루시네이션 의심

4. 전체 테스트 결과 요약:
 번호   카테고리                                       질문  답변 길이                                                                                                        답변 요약
  1 명확한 답변                       당뇨병의 진단 기준은 무엇인가요?    117 \n\n【출처】\n- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)\n- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)\n- 2023 당뇨병 진료지침_전문_...
  2 명확한 답변                당화혈색소(HbA1c)의 목표치는 얼마인가요?    117 \n\n【출처】\n- 2023 당뇨병 진료지침_전문_240620.pdf (청크 1)\n- 2023 당뇨병 진료지침_전문_240620.pdf (청크 0)\n- 2023 당뇨병 진료지침_전문_...
  3 명확한 답변            메트포르민(Metformin)의 작용 기전과 용법은?    117 \n\n【출처】\n- 2023 당뇨병 진료지침_전문_240620.pdf (청크 1)\n- 2023 당뇨병 진료지침_전문_240620.pdf (청크 1)\n- 2023 당뇨병 진료지침_전문_...
  4 명확한 답변               당뇨병성 망막병증의 선별검사는 언제 시행하나요?    5

In [16]:
# ===============================================
# 셀 17: 결과 저장
# ===============================================
import json
from datetime import datetime

# 테스트 결과를 JSON으로 저장
output_dir = "./test_results"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = os.path.join(output_dir, f"qa_test_results_{timestamp}.json")

# 결과 데이터 준비
save_data = {
    "test_date": timestamp,
    "config": {"retriever": RETRIEVER_CONFIG, "llm": LLM_CONFIG},
    "results": test_results,
    "summary": {
        "total_tests": len(test_results),
        "avg_answer_length": float(df_results["답변 길이"].mean()),
        "categories": df_results["카테고리"].value_counts().to_dict(),
    },
}

# JSON 파일로 저장
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(save_data, f, ensure_ascii=False, indent=2)

print(f"\n* 테스트 결과 저장 완료: {output_file}")


* 테스트 결과 저장 완료: ./test_results\qa_test_results_20251201_193647.json
